In [8]:
import pandas as pd
from supabase import create_client, Client
import os
import matplotlib.pyplot as plt

supabase_url = os.getenv('SUPABASE_URL')
supabase_key = os.getenv('SUPABASE_KEY') 

url: str = supabase_url
key: str = supabase_key

supabase: Client = create_client(url, key)

response = supabase.table('produtosKabum').select('*').execute()

df = pd.DataFrame(response.data)
df['id_kabum'] = df['id_kabum'].astype(int).astype(str)

df.head(3)

,id,data,id_kabum,categoria,subcategoria,nome,preco,preco_pix,descricao,openbox,imagem,site,url
0,264,2024-06-19T19:41:29.3192+00:00,128250,Hardware,Disco Rígido (HD),"HD Seagate IronWolf NAS, 14TB, 3.5´, SATA - ST...",3294.11,2799.99,Compre HD Seagate IronWolf NAS no KaBuM! As me...,0,,kabum,https://www.kabum.com.br/produto/128250
1,265,2024-06-19T19:41:29.432631+00:00,100916,Hardware,Disco Rígido (HD),"HD Seagate 2TB BarraCuda, 3.5', SATA - ST2000D...",494.11,419.99,"Um desempenho robusto e confiabilidade, Cache ...",0,https://images.kabum.com.br/produtos/fotos/100...,kabum,https://www.kabum.com.br/produto/100916
2,266,2024-06-19T19:41:29.517339+00:00,95803,Hardware,Disco Rígido (HD),"HD Seagate 4TB BarraCuda, 3.5', SATA - ST4000D...",823.52,699.99,O HD perfeito para armazenamento de todos os s...,0,https://images.kabum.com.br/produtos/fotos/958...,kabum,https://www.kabum.com.br/produto/95803


In [9]:
# ... código existente de importação e conexão ...

def criar_historico_completo(df_original):
    # Criar uma lista para armazenar os DataFrames processados
    dfs_processados = []
    
    # Obter lista única de IDs dos produtos
    produtos_unicos = df_original['id_kabum'].unique()
    
    for id_produto in produtos_unicos:
        # Selecionar dados do produto atual
        produto = df_original[df_original['id_kabum'] == id_produto].copy()
        
        # Converter e extrair apenas a data
        produto['data'] = pd.to_datetime(produto['data'])
        produto['data_somente'] = produto['data'].dt.date
        
        # Remover duplicatas por data
        produto_sem_duplicatas = produto.drop_duplicates(subset='data_somente')
        
        # Criar série de datas completa
        data_inicial = produto_sem_duplicatas['data_somente'].min()
        data_final = produto_sem_duplicatas['data_somente'].max()
        todas_as_datas = pd.date_range(data_inicial, data_final).date
        
        # Criar DataFrame com todas as datas
        df_datas_completas = pd.DataFrame({'data_somente': todas_as_datas})
        
        # Mesclar com dados do produto
        df_produto_completo = df_datas_completas.merge(
            produto_sem_duplicatas, 
            on='data_somente', 
            how='left'
        )
        
        # Preencher valores ausentes
        colunas_para_preencher = ['preco', 'preco_pix', 'nome', 'id_kabum', 'categoria', 'url']
        df_produto_completo[colunas_para_preencher] = df_produto_completo[colunas_para_preencher].ffill()
        
        dfs_processados.append(df_produto_completo)
    
    # Combinar todos os DataFrames processados
    historico_completo = pd.concat(dfs_processados, ignore_index=True)
    return historico_completo

# Criar o histórico completo
df_historico = criar_historico_completo(df)


In [12]:
def criar_ranking_descontos(df_historico):
    # Obter a data mais recente
    data_mais_recente = df_historico['data_somente'].max()
    
    # Calcular a data de 7 dias atrás
    data_7_dias = data_mais_recente - pd.Timedelta(days=7)
    
    # Criar DataFrame para análise
    df_analise = []
    
    for id_produto in df_historico['id_kabum'].unique():
        try:
            # Filtrar dados do produto
            produto = df_historico[df_historico['id_kabum'] == id_produto].copy()
            
            # Verificar se existem dados recentes para o produto
            dados_recentes = produto[produto['data_somente'] == data_mais_recente]
            if dados_recentes.empty:
                continue
                
            # Obter preço atual (mais recente)
            preco_atual = dados_recentes['preco_pix'].iloc[0]
            
            # Verificar dados dos últimos 7 dias
            dados_7_dias = produto[
                (produto['data_somente'] >= data_7_dias) & 
                (produto['data_somente'] < data_mais_recente)
            ]
            
            # Se não houver dados suficientes para comparação, pular
            if len(dados_7_dias) < 3:  # Exigir pelo menos 3 dias de dados
                continue
                
            # Calcular média dos últimos 7 dias
            media_7_dias = dados_7_dias['preco_pix'].mean()
            
            # Calcular desconto percentual
            desconto = int(((media_7_dias - preco_atual) / media_7_dias) * 100)
            
            df_analise.append({
                'id_kabum': str(int(id_produto)),  # Convertendo para int e depois para string para remover o .0
                'nome': produto['nome'].iloc[0],
                'categoria': produto['categoria'].iloc[0],
                'subcategoria': produto['subcategoria'].iloc[0],
                'preco_atual': preco_atual,
                'media_7_dias': media_7_dias,
                'desconto_percentual': desconto,
                'url': produto['url'].iloc[0]
            })
            
        except Exception as e:
            print(f"Erro ao processar produto {id_produto}: {str(e)}")
            continue
    
    # Criar DataFrame com os resultados
    df_descontos = pd.DataFrame(df_analise)
    
    if df_descontos.empty:
        print("Nenhum produto com dados suficientes para análise foi encontrado.")
        return pd.DataFrame()
    
    # Ordenar por desconto (maiores descontos primeiro)
    df_descontos = df_descontos.sort_values('desconto_percentual', ascending=False)
    
    # Arredondar valores numéricos (apenas para preços)
    df_descontos['preco_atual'] = df_descontos['preco_atual'].round(2)
    df_descontos['media_7_dias'] = df_descontos['media_7_dias'].round(2)
    
    return df_descontos

# Criar o ranking de descontos
df_ranking = criar_ranking_descontos(df_historico)

In [13]:
df_ranking.head()

,id_kabum,nome,categoria,subcategoria,preco_atual,media_7_dias,desconto_percentual,url
271,129459,"Processador AMD Ryzen 7 5800X, 3.8GHz (4.7GHz ...",Hardware,Processadores,1699.99,3380.09,49,https://www.kabum.com.br/produto/129459
261,320797,"Processador AMD Ryzen 7 5700X, 3.4GHz (4.6GHz ...",Hardware,Processadores,1099.99,2005.66,45,https://www.kabum.com.br/produto/320797
178,238635,"Aircooler Redragon Tyr, LED Azul, Intel e AMD,...",Hardware,Coolers,84.99,131.42,35,https://www.kabum.com.br/produto/238635
266,181089,"Processador AMD Ryzen 7 5700G, 3.8GHz (4.6GHz ...",Hardware,Processadores,1129.99,1724.28,34,https://www.kabum.com.br/produto/181089
264,129451,"Processador AMD Ryzen 5 5600X, 3.7GHz (4.6GHz ...",Hardware,Processadores,1089.99,1644.28,33,https://www.kabum.com.br/produto/129451


In [14]:
# Primeiro, vamos preparar os dados para inserção
dados_para_insercao = df_ranking.to_dict('records')

# Limpar a tabela existente e inserir novos dados
try:
    # Deletar todos os registros existentes
    supabase.table('topPromo').delete().neq('id_kabum', '0').execute()
    
    # Inserir novos registros
    response = supabase.table('topPromo').insert(dados_para_insercao).execute()
    print("Dados atualizados com sucesso!")
    
except Exception as e:
    print(f"Erro ao atualizar dados: {str(e)}")

Dados atualizados com sucesso!
